In [27]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [28]:
cd "/content/drive//My Drive/Colab/"

/content/drive/My Drive/Colab


In [0]:

from nltk.corpus import stopwords 
import binascii
import numpy as np
import matplotlib.pyplot as plt
import time 
from sklearn.feature_extraction.text import HashingVectorizer
import os
import pickle
from joblib import Parallel, delayed
import multiprocessing as mp
from itertools import repeat
from numba import jit, cuda
import math

# Part A

In [0]:
inputpath = r'Gutenberg/txt'
filenames = [os.path.join(inputpath, file) for file in os.listdir(inputpath)]
corpus = []
i=0
for file in filenames:
    with open(file) as f:
        try:
            data = " ".join(line.strip() for line in f)
            corpus.append(data)
        except:
            print(file)

Gutenberg/txt/Henry David Thoreau___A Week on the Concord and Merrimack Rivers.txt


In [0]:
pickle.dump( corpus, open( "corpus.p", "wb" ) )

In [0]:
corpus = pickle.load( open( "corpus.p", "rb" ) )

In [0]:
corpus[0][100]

' '

In [0]:
vectorizer = HashingVectorizer(n_features=2**18,ngram_range=(2,2), binary=True, norm = None)
X = vectorizer.fit_transform(corpus)



In [0]:
vectorizer = HashingVectorizer(input='content', n_features=2**18,ngram_range=(2,2), binary=True, norm = None)
X2 = vectorizer.fit_transform()

TypeError: ignored

In [0]:
pickle.dump( X, open( "X.p", "wb" ) )

In [0]:
X = pickle.load( open( "X.p", "rb" ) )

In [0]:
X.toarray()[923,:].sum()

15561.0

In [0]:
X.shape

(3066, 262144)

In [0]:
%%time
#########Generate Minhash Signature for each book#####
@jit(nopython=True, parallel=True)
def parrallel_minhash(X):
  maxShingleID = pow(2,32)
  prime = 4294967311
  numHashFunctions = 90
  # c = []
  c = np.random.uniform(0,prime-1,2*numHashFunctions)
  c1 = c[0:numHashFunctions]
  c2 = c[numHashFunctions:2*numHashFunctions]

  minhashSignatures = np.zeros((X.shape[0],numHashFunctions))

  for book in range(X.shape[0]):
      shingleIDs = X[book]
      minhashSignature = np.zeros(numHashFunctions)

      for i in range(0,numHashFunctions):
          minhashValue = math.inf
          for shingleID in np.nditer(shingleIDs):
              hashValue = (c1[i]*shingleID+c2[i])%prime
              if hashValue<minhashValue:
                  minhashValue = hashValue
          minhashSignature[i]=minhashValue

      minhashSignatures[book,:] = minhashSignature
      print(book)

  return minhashSignatures

minhashSignatures = parrallel_minhash(X.toarray())

In [0]:
pickle.dump( minhashSignatures, open( "minhashSignatures.p", "wb" ) )

In [0]:
minhashSignatures= pickle.load( open( "SIG.pkl", "rb" ) )

In [0]:
minhashSignatures.shape

(3066, 90)

In [32]:
%%time
# rows = [1,3,5,7,9]
# bands = [1,3,5,7,9]
# band = bands[0]
row = 3
band = 30

# for row in rows: 
#     for count in range(0,10):


numHashFunctions = band*row
S = {}
key = []
# minhashSignatures = minhashSignatures
IDs = list(range(0,minhashSignatures.shape[0]))
for m in IDs:
    key.append(m)
    fp = []
    minhashSignatureArticle1 = minhashSignatures[IDs.index(m)]
    minhashSignatureArticle1Chuncks = np.array_split(np.array(minhashSignatureArticle1),band)

    for ID in range(m,len(IDs)):

        minhashSignatureArticlei = minhashSignatures[IDs.index(ID)]
        minhashSignatureArticleiChuncks = np.array_split(np.array(minhashSignatureArticlei),band)


        for i in range(0,len(minhashSignatureArticle1Chuncks)):
            if np.all(minhashSignatureArticle1Chuncks[i] == minhashSignatureArticleiChuncks[i]) and m!=ID:

                try:
                    S[m].append(ID)
                except:
                    S[m] = [ID]
                # IDs.remove(ID)
                
                
                break;
#         print(S)

    print(m)
    fpnum = []
    for i in S:
        article1 = set(minhashSignatureArticle1)
        articlei = set(S[i])
        trueJaccard = len(article1.intersection(articlei))/len(article1.union(articlei))
#             print(trueJaccard)
        if trueJaccard<0.8:

            fpnum.append(trueJaccard)

#         print(fpnum)
    fp.append(len(fpnum)/len(S))

0
1
2
3
4
5
6
7


KeyboardInterrupt: ignored

In [0]:
pickle.dump( S, open( "S.p", "wb" ) )

In [0]:
S = pickle.load( open( "S.p", "rb" ) )

In [0]:
del S[3037]

In [0]:

approxJaccard = np.zeros((X.shape[0],X.shape[0]))
trueJaccard = np.zeros((X.shape[0],X.shape[0]))
for key in S:
  minhashSignatureArticle1 = minhashSignatures[key]
  # print(minhashSignatureArticle1)
  # print(key)
  article1 = set(minhashSignatureArticle1)
  # print(article1)
  for second in S[key]:
        # print(second)
        minhashSignatureArticlei = minhashSignatures[second]
        articlei = set(minhashSignatureArticlei)
        # approxJaccard[key,second] = len(np.where(minhashSignatureArticlei == minhashSignatureArticle1))/len(minhashSignatureArticle1)
        approxJaccard[key,second] =len(article1.intersection(articlei))/len(article1)
        trueJaccard[key,second] = len(article1.intersection(articlei))/len(article1.union(articlei))
        

In [0]:
trueJaccard[ 766,  788]

1.0

In [0]:
trueJaccard[1,:]

array([0., 0., 1., ..., 1., 1., 1.])

In [0]:
def k_largest_index_argsort(a, k):
    idx = np.argsort(a.ravel())[:-k-1:-1]
    return np.column_stack(np.unravel_index(idx, a.shape)) , 

k_largest_index_argsort(approxJaccard, k=10)   

(array([[ 766,  788],
        [1930, 2047],
        [1930, 2049],
        [1930, 2050],
        [1930, 2051],
        [1930, 2052],
        [1930, 2053],
        [1930, 2054],
        [1930, 2055],
        [1930, 2056]]),)

In [0]:
k_largest_index_argsort(trueJaccard, k=10) 

(array([[ 766,  788],
        [1930, 2047],
        [1930, 2049],
        [1930, 2050],
        [1930, 2051],
        [1930, 2052],
        [1930, 2053],
        [1930, 2054],
        [1930, 2055],
        [1930, 2056]]),)

In [0]:
inputpath = r'./Gutenberg/txt'
filenames = [os.path.join(inputpath, file) for file in os.listdir(inputpath)]
filenames[1490]

"./Gutenberg/txt/Anthony Trollope___The Kellys and the O'Kellys.txt"

# Part C

In [0]:
X.shape[0]

3066

In [0]:
@jit(nopython=True, parallel=True)
def parrallel_jaccard(X):
  S = X
  AllExact = np.zeros((S.shape[0],S.shape[0]))

  for i in range(0,len(AllExact)):
      for j in range(i+1,len(AllExact)):
              article1 = set(X[i,:])
              articlei = set(X[j,:])

              # ExacttrueJaccard = len(np.where(X[i,:] == X[j,:]))/len(np.unique(np.concatenate(X[i,:], X[j,:])))
              ExacttrueJaccard = len(article1.intersection(articlei))/len(article1.union(articlei))
              AllExact[i,j] = ExacttrueJaccard
      print(i)
  # print(All)
  return AllExact    

In [0]:
%%time
trueSim = parrallel_jaccard(X.toarray())